In [1]:
import pandas as pd
import os
import re

# Functions

In [2]:
def transform_waves(s):
    # Remove spaces
    s = s.replace(" ", "")
    
    # Split on commas
    parts = s.split(",")
    
    result = []
    
    for part in parts:
        # Match either wN or wN-M
        m = re.match(r"w(\d+)(?:-(\d+))?", part)
        if m:
            start = int(m.group(1))
            end = int(m.group(2)) if m.group(2) else start
            result.extend(range(start, end+1))
    return result


def expand_variable_rows(row):
    waves_list = row["WavesList"]
    variable = row["Variable"]
    
    if "w" in variable:
        # variable has wave pattern
        expanded = []
        for w in waves_list:
            # replace 'w' + digits with the wave number
            new_variable = re.sub(r"w\d+", f"{w}", variable)
            expanded.append({
                "Topic": row["Topic"],
                "Variable": new_variable,
                "Description": row["Description"],
                "Dataset": row["Dataset"],
                "Wave": w
            })
        return expanded
    else:
        # keep as is, with Wave = None
        return [{
            "Topic": row["Topic"],
            "Variable": variable,
            "Description": row["Description"],
            "Dataset": row["Dataset"],
            "Wave": None
        }]
    

def rename_columns(df, country):
    cols = df.columns.tolist()
    new_cols = []
    
    if country == 'india':
        # Replace any prefix ending with 1 → 5
        for c in cols:
            m = re.match(r'^(s|r|h|hh)1(.*)', c)
            if m:
                new_c = f"{m.group(1)}5{m.group(2)}"
                new_cols.append(new_c)
            else:
                new_cols.append(c)

    elif country == 'mexico':
        for c in cols:
            m = re.match(r'^(s|r|h|hh)([1-4])(.*)', c)
            if m:
                old_wave = int(m.group(2))
                new_wave = old_wave + 1
                new_c = f"{m.group(1)}{new_wave}{m.group(3)}"
                new_cols.append(new_c)
            else:
                new_cols.append(c)

    elif country == 'uk':
        wave_map = {
            '8': '5',
            '7': '4',
            '6': '3',
            '5': '2',
            '4': '1'
        }
        for c in cols:
            m = re.match(r'^(s|r|h|hh)([4-8])(.*)', c)
            if m:
                new_wave = wave_map[m.group(2)]
                new_c = f"{m.group(1)}{new_wave}{m.group(3)}"
                new_cols.append(new_c)
            else:
                new_cols.append(c)

    elif country == 'us':
        wave_map = {
            '13': '5',
            '12': '4',
            '11': '3',
            '10': '2',
            '9': '1'
        }
        for c in cols:
            m = re.match(r'^(s|r|h|hh)(13|12|11|10|9)(.*)', c)
            if m:
                new_wave = wave_map[m.group(2)]
                new_c = f"{m.group(1)}{new_wave}{m.group(3)}"
                new_cols.append(new_c)
            else:
                new_cols.append(c)

    else:
        # no renaming for other countries yet
        new_cols = cols

    df.columns = new_cols
    return df


def select_columns(df, country, allowed_digits):
    """
    Select columns in df that:
    - either start with s/r/h/hh followed by one of the allowed digits
    - or contain no digits at all
    """
    digits_pattern = "|".join(sorted(allowed_digits[country], key=len, reverse=True))
    # Build regex for allowed prefixes + digits
    regex_pattern = re.compile(rf"^(s|r|h|hh)({digits_pattern})")

    keep_cols = []
    for col in df.columns:
        if regex_pattern.match(col):
            keep_cols.append(col)
        elif not re.search(r"\d", col):
            keep_cols.append(col)
    
    return df[keep_cols]

# Variables selection

In [3]:
# table_variables = pd.read_excel('../data/core/comparision/common_variables_short.xlsx')
# table_variables['Topic'] = table_variables['Topic'].fillna(method="ffill")
# table_variables['Variable'] = table_variables['Variable'].fillna(method="ffill")
# table_variables['Description'] = table_variables['Description'].fillna(method="ffill")
# table_variables = table_variables[table_variables['Dataset'].notna()]
# table_variables = table_variables[table_variables['Dataset'].apply(
#     lambda x: ('HRS' in x) or ('MHAS' in x) or ('ELSA' in x) or ('LASI' in x))]
# table_variables['Waves'] = table_variables['Dataset'].str.split(' - ').str[1]
# table_variables['Dataset'] = table_variables['Dataset'].str.split(' - ').str[0]
# table_variables['VariableNormalized'] = table_variables['Variable'].str.replace('Rw', '').str.lower()

# table_variables.to_excel('../data/processed/table_variables.xlsx', index=False)

In [4]:
table_variables = pd.read_excel('../data/processed/table_variables.xlsx')

In [5]:
# Replace Variable with AlternativeVariable if AlternativeVariable is not None
table_variables['Variable'] = table_variables['AlternativeVariable'].where(table_variables['AlternativeVariable'].notnull(), table_variables['Variable'])
table_variables = table_variables.drop(columns=['AlternativeVariable'])
table_variables['WavesList'] = table_variables['Waves'].apply(lambda x: transform_waves(x))

# Apply to each row and flatten the results
expanded_rows = []
for _, row in table_variables.iterrows():
    expanded_rows.extend(expand_variable_rows(row))

# Create new DataFrame
table_variables_waves = pd.DataFrame(expanded_rows)

mask = table_variables_waves["Wave"].notnull()
table_variables_waves.loc[mask, "Variable"] = table_variables_waves.loc[mask].apply(
    lambda row: row["Variable"].replace("w", str(int(row["Wave"]))),
    axis=1
)

table_variables_waves['Variable'] = table_variables_waves['Variable'].str.lower()

# 1. India

In [6]:
df_lasi = pd.read_stata('../data/core/LASI/Harmonized LASI A.3_Stata/H_LASI_a3.dta')
df_lasi_dad = pd.read_stata('../data/hcap/LASI/Harmonized_LASI-DAD_Ver_B.1/H_LASI_DAD_b1.dta')

In [ ]:
df_lasi_dad = df_lasi_dad[['prim_key', 'r1sgcp', 'r2sgcp']]
df_india = df_lasi.merge(df_lasi_dad, on='prim_key', how='inner')

columns_india = table_variables_waves[table_variables_waves['Dataset'] == 'Harmonized LASI']['Variable'].tolist()
# score_columns_india = ['r1sgcp', 'r2sgcp']
# system_columns_india = ['prim_key', 'hhid', 'pnc', 'pn', 'h1coupid', 's1prim_key']
score_columns_india = ['r1sgcp']
system_columns_india = ['prim_key']

selected_columns_india = list(set(df_india.columns) & set(columns_india))
selected_columns_india = system_columns_india + selected_columns_india + score_columns_india

df_india = df_india[selected_columns_india]

# Convert all categorical columns to strings
for col in df_india.columns:
    if pd.api.types.is_categorical_dtype(df_india[col]):
        df_india[col] = df_india[col].astype(str)

df_india = df_india.rename(columns={'r1sgcp': 'fgcp', 'prim_key': 'id_resp'})
df_india['country'] = 'india'

df_india = df_india[df_india['fgcp'].notna()]

df_india = df_india.rename(columns={'hh5rural': 'h5rural'})
df_india = df_india.rename(columns={'r5coresd': 'h5coresd'})

In [8]:
df_india.to_parquet('../data/processed/df_india.parquet')

# 2. US

In [9]:
df_hrs_harmonized = pd.read_spss('../data/core/HRS/H_HRS_d_spss/H_HRS_d.sav')
df_hrs_rand = pd.read_stata('../data/core/HRS/randhrs1992_2022v1_STATA/randhrs1992_2022v1.dta')
df_hrs_eol = pd.read_stata('../data/core/HRS/HarmonizedHRSEndOfLifeA/H_HRS_EOL_a.dta')
df_hrs_scores = pd.read_stata(
    '../data/hcap/HRS/HCAP-Harmonized-Factor-Scores/interim-HcapHarmoniz-305-v2-20230725-hrshcap.dta')

In [ ]:
df_hrs_harmonized['hhidpn'] = df_hrs_harmonized['hhidpn'].astype(int)

# Rename 'agey_b' to 'agey' in columns
df_hrs_rand = df_hrs_rand.rename(columns=lambda x: x.replace('agey_b', 'agey'))

df_hrs_scores = df_hrs_scores.drop(columns=['study'])
df_hrs_scores = df_hrs_scores.rename(columns={'id_hrs': 'hhidpn'})

df_hrs_harmonized['hhidpn'] = df_hrs_harmonized['hhidpn'].astype(int)
df_hrs_rand['hhidpn'] = df_hrs_rand['hhidpn'].astype(int)
df_hrs_eol['hhidpn'] = df_hrs_eol['hhidpn'].astype(int)
df_hrs_scores['hhidpn'] = df_hrs_scores['hhidpn'].astype(int)

df_us = df_hrs_harmonized.merge(df_hrs_rand, on=['hhidpn', 'hhid', 'pn'], how='left')
df_us = df_us.merge(df_hrs_eol, on=['hhidpn', 'hhid', 'pn'], how='left')
df_us = df_us.merge(df_hrs_scores, on='hhidpn', how='inner')

datasets_us = set(['Harmonized HRS', 'RAND HRS Longitudinal File', 'Harmonized HRS EOL'])
columns_us = table_variables_waves[table_variables_waves['Dataset'].isin(datasets_us)]['Variable'].tolist()
score_columns_us = ['fgcp']
system_columns_us = ['hhidpn']

selected_columns_us = list(set(df_us.columns) & set(columns_us))
selected_columns_us = system_columns_us + selected_columns_us + score_columns_us
df_us = df_us[selected_columns_us]

# Convert all categorical columns to strings
for col in df_us.columns:
    if pd.api.types.is_categorical_dtype(df_us[col]):
        df_us[col] = df_us[col].astype(str)

df_us = df_us.rename(columns={'hhidpn': 'id_resp'})
df_us['country'] = 'us'

# Rename columns
df_us = df_us.rename(columns={
    'r1doctor': 'r1doctor1y',
    'r2doctor': 'r2doctor1y',
    'r3doctor': 'r3doctor1y',
    'r4doctor': 'r4doctor1y',
    'r5doctor': 'r5doctor1y'
})

df_us = df_us.rename(columns={
    'r1sayret': 'r1retemp',
    'r2sayret': 'r2retemp',
    'r3sayret': 'r3retemp',
    'r4sayret': 'r4retemp',
    'r5sayret': 'r5retemp'
})

df_us = df_us.rename(columns={
    'rassrecv': 'r1pubpen'
})

df_us['r2pubpen'] = df_us['r1pubpen']
df_us['r3pubpen'] = df_us['r1pubpen']
df_us['r4pubpen'] = df_us['r1pubpen']
df_us['r5pubpen'] = df_us['r1pubpen']

In [11]:
df_us.to_parquet('../data/processed/df_us.parquet')

# 3. UK

In [12]:
df_elsa = pd.read_stata('../data/core/ELSA/UKDA-5050-stata/stata/stata13_se/h_elsa_g3.dta')
df_elsa_scores = pd.read_spss('../data/hcap/ELSA/spss/spss28/hcap_2018_harmonised_scores.sav')

In [ ]:
df_elsa_scores['id_elsa'] = df_elsa_scores['id_elsa'].astype(int)
df_elsa_scores = df_elsa_scores.drop(columns=['study'])
df_uk = df_elsa.merge(df_elsa_scores, left_on='idauniq', right_on='id_elsa', how='inner')

columns_uk = table_variables_waves[table_variables_waves['Dataset'] == 'Harmonized ELSA']['Variable'].tolist()
score_columns_uk = ['fgcp']
system_columns_uk = ['id_elsa']

selected_columns_uk = list(set(df_elsa.columns) & set(columns_uk))
selected_columns_uk = system_columns_uk + selected_columns_uk + score_columns_uk

df_uk = df_uk[selected_columns_uk]

# Convert all categorical columns to strings
for col in df_uk.columns:
    if pd.api.types.is_categorical_dtype(df_uk[col]):
        df_uk[col] = df_uk[col].astype(str)

df_uk = df_uk.rename(columns={'id_elsa': 'id_resp'})
df_uk['country'] = 'uk'

# Remove '_e' from columns
df_uk = df_uk.rename(columns=lambda x: x.replace('_e', ''))


In [14]:
df_uk.to_parquet('../data/processed/df_uk.parquet')

# 4. Mexico

In [15]:
df_mhas = pd.read_spss('../data/core/MHAS/H_MHAS_c2.sav')
df_mexcog = pd.read_stata('../data/hcap/Mex-Cog/Multi-Country Harmonized Factor Scores Mex-Cog 2016.dta')

In [ ]:
df_mhas['cunicah'] = df_mhas['cunicah'].astype(int)
df_mhas['np'] = df_mhas['np'].astype(int)
df_mexcog['cunicah'] = df_mexcog['cunicah'].astype(int)
df_mexcog['np'] = df_mexcog['np'].astype(int)

df_mexico = df_mhas.merge(df_mexcog, on=['cunicah', 'np'], how='inner')

columns_mexico = table_variables_waves[table_variables_waves['Dataset'] == 'Harmonized MHAS']['Variable'].tolist()
score_columns_mexico = ['fgcp']
system_columns_mexico = ['unhhidnp']

selected_columns_mexico = list(set(df_mexico.columns) & set(columns_mexico))
selected_columns_mexico = system_columns_mexico + selected_columns_mexico + score_columns_mexico

df_mexico = df_mexico[selected_columns_mexico]

# Convert all categorical columns to strings
for col in df_mexico.columns:
    if pd.api.types.is_categorical_dtype(df_mexico[col]):
        df_mexico[col] = df_mexico[col].astype(str)

df_mexico = df_mexico.rename(columns={'unhhidnp': 'id_resp'})
df_mexico['country'] = 'mexico'

# Remove '_m' from columns
df_mexico = df_mexico.rename(columns=lambda x: x.replace('_m', ''))

df_mexico.to_parquet('../data/processed/df_mexico.parquet')

# Combine datasets

## Used waves

### 1. India

- X waves: 1 (2017–2019)
- Y: 2017-2019

### 2. US

- X waves:

    9 (2008–2009)

    10 (2010–2011)

    11 (2012–2013)

    12 (2014–2015)

    13 (2016–2018)

- Y: 2016

### 3. UK

- X waves:

    4 May 2008 – Jul 2009

    5 Jun 2010 – Jul 2011

    6 May 2012 – Jun 2013

    7 Jun 2014 – May 2015

    8 May 2016 – Jun 2017

- Y: 15 January 2018 and 8 April 2018

### 4. Mexico

- X waves:

	1 - 2001

	2 - 2003

	3 - 2012

	4 - 2015

- Y: 2016



In [17]:
df_india = pd.read_parquet('../data/processed/df_india.parquet')
df_us = pd.read_parquet('../data/processed/df_us.parquet')
df_uk = pd.read_parquet('../data/processed/df_uk.parquet')
df_mexico = pd.read_parquet('../data/processed/df_mexico.parquet')

In [18]:
df_india['id_resp'] = df_india['country'] + '_' + df_india['id_resp'].astype(int).astype(str)
df_us['id_resp'] = df_us['country'] + '_' + df_us['id_resp'].astype(int).astype(str)
df_uk['id_resp'] = df_uk['country'] + '_' + df_uk['id_resp'].astype(int).astype(str)
df_mexico['id_resp'] = df_mexico['country'] + '_' + df_mexico['id_resp'].astype(int).astype(str)

# Map of allowed digits for each country
allowed_waves = {
    "us": ['9', '10', '11', '12', '13'],
    "uk": ['4', '5', '6', '7', '8'],
    "mexico": ['1', '2', '3', '4'],
    "india": ['1']
}

df_india = select_columns(df_india, 'india', allowed_waves)
df_us = select_columns(df_us, 'us', allowed_waves)
df_uk = select_columns(df_uk, 'uk', allowed_waves)
df_mexico = select_columns(df_mexico, 'mexico', allowed_waves)

df_us = rename_columns(df_us, 'us')
df_uk = rename_columns(df_uk, 'uk')
df_mexico = rename_columns(df_mexico, 'mexico')
df_india = rename_columns(df_india, 'india')

df_mexico = df_mexico.drop(columns=['raedyrs'])
df_us = df_us.drop(columns=['raedyrs'])
df_india = df_india.drop(columns=['raedyrs'])
df_uk = df_uk.drop(columns=['raedyrs'])

all_columns = sorted(set(df_us.columns) | set(df_uk.columns) | set(df_mexico.columns) | set(df_india.columns))

df_us = df_us.reindex(columns=all_columns)
df_uk = df_uk.reindex(columns=all_columns)
df_mexico = df_mexico.reindex(columns=all_columns)
df_india = df_india.reindex(columns=all_columns)

df_combined = pd.concat([df_us, df_uk, df_mexico, df_india])

df_combined["id_resp"] = df_combined["id_resp"].astype(str)

/var/folders/k2/l43pc0gj6311hvl_9yxc3kzc0000gn/T/ipykernel_28863/127253590.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_combined = pd.concat([df_us, df_uk, df_mexico, df_india])


In [19]:
# def compute_notna_countries(df_combined):

#     # Compute % missing for each country
#     result = pd.DataFrame()

#     for name, group in df_combined.groupby("country"):
#         pct_missing = 100 * group.notna().sum() / len(group)
#         pct_missing = pct_missing.drop("country", errors="ignore")
#         result[name] = pct_missing

#     # Optional formatting
#     result = result.round(1)

#     # Put variables as a column instead of index if desired
#     result = result.reset_index()
#     result = result.rename(columns={'index': 'variable'})

#     return result


# notna_countries = compute_notna_countries(df_combined)
# notna_countries = notna_countries.set_index('variable')
# notna_countries = notna_countries > 0

# notna_countries['sum'].value_counts(normalize=True)

In [20]:
df_combined = df_combined.set_index('id_resp')

In [26]:
df_combined = pd.read_pickle('../data/processed/df_combined.pkl')

In [30]:
cols_to_drop = [
    "r1doctor",
    "r1sayret",
    "r2doctor",
    "r2sayret",
    "r3doctor",
    "r3sayret",
    "r4doctor",
    "r4sayret",
    "r5coresd",
    "r5doctor",
    "r5sayret",
    'hh5rural'
]

df_combined = df_combined.drop(columns=cols_to_drop, errors="ignore")

In [32]:
# Save to pickle
df_combined.to_pickle('../data/processed/df_combined.pkl')